In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import time
import re
import requests
from urllib.request import urlopen
import bs4
import warnings

In [4]:
import sys
cheminMonGitHub = '/content/drive/MyDrive/ProjetML/Base_BookCrossing/'
sys.path.append(cheminMonGitHub)

In [5]:
import Parser_csv_BooksList as P
import BooksSearch_GoodReads as GR
import BooksSearch_Google as G
import BooksSearch_BothWebSites as BothWS

In [42]:
import importlib as imp
imp.reload(P)	
imp.reload(GR)	
imp.reload(G)
imp.reload(BothWS)

<module 'BooksSearch_BothWebSites' from '/content/drive/MyDrive/ProjetML/Base_BookCrossing/BooksSearch_BothWebSites.py'>

In [43]:
def internetSearch(SaveFileName, theColumns, ind_start, ind_end):

    NotFoundBooks_ind = 0   
    NotFoundBooks_bool = True
    
    pdT = pd.DataFrame(columns = theColumns)

    ret, refBookGoogle = G.GoogleSearchFunction(isbn = booksDF.isbn[ind_start])
    pdT.loc[0] = refBookGoogle 
    pdT.loc[0:1].to_csv(SaveFileName, sep = ';', index = False, mode = 'w')

    for i in range(ind_start+1, ind_end):

      #First we try to found this book on our existing data base
      if not (any(booksBase.isbn == booksDF.isbn[i])):
        if not (any(booksDuplicates.isbn == booksDF.isbn[i])):
          #ISBN not found on our existing data base
  
          if len(booksBase[ booksBase.book_title == booksDF.original_title[i] ].book_author[booksBase.book_author == booksDF.authors[i].split(',')[0]]) == 0:
            if len(booksDuplicates[ booksDuplicates.book_title == booksDF.original_title[i] ].book_author[booksDuplicates.book_author == booksDF.authors[i].split(',')[0]]) == 0:
              #Title + Author not found on our existing data base
              
              retGoogle = False
              retGoodReads = False
              retPartial = False

              #First we use Google Books web site: search fastest
              retGoogle, refBookGoogle = G.GoogleSearchFunction(isbn = booksDF.isbn[i])
              #print("livre {0} found on Google: {1}".format(booksDF.isbn[i], retGoogle))

              if (retGoogle == False):

                  #if ISBN not found on Google Books, we try on Google Reads
                  retGoodReads, refBookGoodReads = GR.GoodReadsSearchFunction(isbn = booksDF.isbn[i])
                  #print("livre {0} found on GoodReads: {1}".format(booksDF.isbn[i], retGoodReads))

                  if (retGoodReads == False):
                      #if ISBN not found (on Google Books and on Good Reads), 
                      #we try search on GoodReads only a search with title and author 
                      retGoodReads, refBookGoodReads = GR.GoodReadsSearchFunction(title = booksDF.original_title[i], author = booksDF.authors[i].split(',')[0])
                      if (retGoodReads == True):
                          refBook = refBookGoodReads
                          NotFoundBooks_bool = False
                      else:
                          NotFoundBooks_ind += 1
                  else:
                      refBook = refBookGoodReads
                      NotFoundBooks_bool = False
                      
              else:
                  refBook = refBookGoogle
                  NotFoundBooks_bool = False
                  
                  #If ISBN found on Google Books, we complete some columns with GoodReads search
                  retPartial, refBookPartial = BothWS.GoodReadsPartialSearchFunction(isbn = booksDF.isbn[i])
                  #print("livre {0} found on Partial GoodReads: {1}".format(booksDF.isbn[i], retPartial))
                  #print("\nrefBookPartial\n", refBookPartial)
                  if (retPartial == True):
                      refBook[-4:] = refBookPartial
                  #print("\nrefBook\n", refBook)

              if NotFoundBooks_bool != True:
                  pdT.loc[0] = refBook
                  pdT.loc[0:1].to_csv(SaveFileName, sep = ';', index = False, mode = 'a', header = False)
                  NotFoundBooks_bool = True
        
    return NotFoundBooks_ind

In [45]:
cheminBaseOriginale = '/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_originale/'
cheminCommon = '/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/'

SaveFileName = cheminCommon + 'bothWebSites_InternetSearch_AllBooks_GoodBooks.csv'
        
theColumns = ['ISBN_10', 'ISBN_13', 'OtherID', 'Book-Title', 'Book-Author', \
                            'Year-Of-Publication', 'Publisher', 'Category', 'Description', 'Language', \
                            'Image', 'Pages', 'Awards', "Author's genre", 'Same serie', 'average_rating']
    
#List of books to search on Google / GoodReads
booksDF = pd.read_csv(cheminBaseOriginale + 'books.csv', dtype = {"isbn13": float})
booksBase = pd.read_csv(cheminCommon + 'bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned.csv', sep = '|', dtype = {"isbn13": float})
booksDuplicates = pd.read_csv(cheminCommon + 'Books_Duplicates.csv', dtype = {"isbn13": float}, sep = '|')


##### Tests manipulations DF

In [7]:
booksDF.head(1)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [8]:
booksDF.isbn13[0]

9780439023480.0

In [9]:
booksBase.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


In [10]:
booksDuplicates.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,345465083,"9,78E+12",0.0,Seabiscuit : An American Legend,Laura Hillenbrand,2003-01-01,"Random House Digital, Inc.",histori,NaN,"Retraces the journey of Seabiscuit, a horse wi...",en,http://books.google.com/books/content?id=kAvhr...,453.0,histori,NaN,[],National Book Critics Circle Award Nominee for...


In [ ]:
any(booksBase.isbn == booksBase.isbn[0])

True

In [ ]:
booksBase[["book_title", "book_author"]] == ("Classical Mythology", "Mark P. O. Morford")

,book_title,book_author
0,True,True
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
99537,False,False
99538,False,False
99539,False,False
99540,False,False


In [ ]:
booksBase[ booksBase.book_title == "Classical Mythology" ].book_author

0    Mark P. O. Morford
Name: book_author, dtype: object

In [ ]:
len(booksBase[ booksBase.book_title == "Classical Mythology" ].book_author[booksBase.book_author == "Mark P. O. Morford"])

1

In [ ]:
len(booksBase[ booksBase.book_title == "Tutu" ].book_author[booksBase.book_author == "Mark P. O. Morford"])

0

In [ ]:
(booksBase[["book_title", "book_author"]] == ("Classical Mythology", "Mark P. O. Morford")).values == [True, True]

array([[ True,  True],
       [False, False],
       [False, False],
       ...,
       [False, False],
       [False, False],
       [False, False]])

In [ ]:
(booksBase[["book_title", "book_author"]] == ("Classical Mythology", "Mark P. O. Morford")).values

array([[ True,  True],
       [False, False],
       [False, False],
       ...,
       [False, False],
       [False, False],
       [False, False]])

In [ ]:
nb_a_trouver = 0

#search books of booksDF inside booksBase and booksDuplicates
for i in range(len(booksDF)):
  #search by comparing ISBN
  if not (any(booksBase.isbn == booksDF.isbn[i])):
    if not (any(booksDuplicates.isbn == booksDF.isbn[i])):
      
      #ISBN not found inside booksBase and booksDuplicates 
      #=> search according to (book_title, book_author)
      if len(booksBase[ booksBase.book_title == booksDF.original_title[i] ].book_author[booksBase.book_author == booksDF.authors[i].split(',')[0]]) == 0:
        if len(booksDuplicates[ booksDuplicates.book_title == booksDF.original_title[i] ].book_author[booksDuplicates.book_author == booksDF.authors[i].split(',')[0]]) == 0:
          
          nb_a_trouver = nb_a_trouver + 1

In [ ]:
nb_a_trouver

6719

##### Lancement Recherche

In [ ]:
#Google search function
NotFoundBooks = internetSearch(SaveFileName, theColumns, 0, len(booksDF)) 
    
print("Not found books: %d" % NotFoundBooks)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. 